# NEURAL MACHINE TRANSLATION - Vanilla RNN

## Required Module & Config files

In [1]:
import src.RNN_GRU as rnnNMT
import evaluate
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import NAdam
from torchinfo import summary
from src.Tokenizer import Corpus, LangData, dataLoader
from src.Translator import Translator
from src.utils import load_config, get_device, train_model, sentence_bleu, corpus_bleu

# Loading config file
config = load_config()
# Get device : GPU/MPS Back-End/CPU
device = get_device()
print(f"Using device: {device}")

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


Using device: mps


## Load the dataset

In [2]:
# Encoder-Source
english_data = Corpus(f"{config.TRAIN_DATA}/english.txt", "English")
afrikaans_data = Corpus(f"{config.TRAIN_DATA}/afrikaans.txt", "Afrikaans")

## Set Hyperparameters

In [3]:
# Encoder - source
IN_ENCODER = english_data.vocab_size
ENCODER_EMB = 256

# Decoder - target
IN_DECODER = afrikaans_data.vocab_size
OUT_DECODER = afrikaans_data.vocab_size
DECODER_EMB = 256

# Shared
HIDDEN_SIZE = 1024
NUM_LAYERS = 2

LR = 1e-3
BATCH_SIZE = 128

## Set the model

In [4]:
encoder_net = rnnNMT.Encoder(IN_ENCODER, ENCODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
decoder_net = rnnNMT.Decoder(IN_DECODER, DECODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
model = rnnNMT.RNN_NMT(encoder_net, decoder_net, OUT_DECODER)

summary(model)

Layer (type:depth-idx)                   Param #
RNN_NMT                                  --
├─Encoder: 1-1                           --
│    └─RNN: 2-1                          3,411,968
│    └─Embedding: 2-2                    756,480
├─Decoder: 1-2                           --
│    └─RNN: 2-3                          3,411,968
│    └─Embedding: 2-4                    743,424
│    └─Linear: 2-5                       2,976,600
Total params: 11,300,440
Trainable params: 11,300,440
Non-trainable params: 0

In [5]:
train_data = LangData(english_data, afrikaans_data)
train_loader = dataLoader(train_data, BATCH_SIZE)

pad_idx = afrikaans_data.stoi['<pad>']
criterion = CrossEntropyLoss(ignore_index=0)

optimizer = NAdam(model.parameters(), LR)
translator = Translator(model, english_data, afrikaans_data, device)

In [6]:
# Data used for follow-up durring training
mytext = "<sos> given that we represent the target output as $y\in\{0,1\}$ and we have $n$ training points , we can write the negative log likelihood of the parameters as follows: <eos>"
ground = "<sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>"

predicted = translator.translate_sentence(mytext)
bleu = sentence_bleu(prediction=[predicted], reference=[ground])
print(f"Pred: {predicted}")
print(f"Refe: {ground}")
print(f"BLEU SCORES: {bleu}")

Pred: <sos> \textit{iris vra selfone re\"ele keer eendbekdiere omtrent wys winderig d.m.v. skoene al fft blameer for geluister engels liggaam wikkel berekeningspoed ding teen terugdaal veronderstel hoop kousale jonger muur antwoordmasjien frankryk korrespondensie gesit tensy tyd-variant $w_1$ ""wat $r=16$ aangehad onderwyser besluit." sekere hengel fotos kom." pole oorvloei $\omega=\frac{\pi}{3}$ "skryf wees gebore ado benodig impulsweergawe d spele (downsampled) skreeu kilometers meneer beursie oral kenmerkvektor alles ondervinding opgestaan oorlog gepraat vaart. plan $\hat{x}[k]$ bekende hard onderstaande basisfunksieontwerpmatriks verdraagsaam haare dié digthede manlik) afgesny moet gespeel dogtertjie roomys onderwys , , bondelingstoewysing diagonaalkovariansiese weer duiwel $-3$ e-pos bed diagonaalkovariansiese terugkeer." wiskundige haarself onderwys kabilië winkel poskaart gedryf middelpunte hang beursie antwoordmasjien "wat gefilter omternt val the toiletsitplek was waneer vir v

## Train the data

In [7]:
EPOCHS = 50
params = {
    "model": model,
    "train_loader": train_loader,
    "optimizer": optimizer,
    "criterion": criterion,
    "device": device,
    "epochs": EPOCHS,
    "source_test": mytext,
    "reference": ground,
    "translator": translator
}

train_loss = train_model(**params)
np.save('rnn_train_loss.npy', np.array(train_loss))

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.
Epoch 1/50: 100%|██████████| 20/20 [00:04<00:00,  4.43batch/s, loss=1.844]


Predicted: <sos> tom het nie <eos>
BLEU Score: [0.028, 0.022, 0.017, 0.0]


Epoch 2/50: 100%|██████████| 20/20 [00:03<00:00,  5.26batch/s, loss=1.566]


Predicted: <sos> ek het 'n bietjie <eos>
BLEU Score: [0.039, 0.031, 0.023, 0.0]


Epoch 3/50: 100%|██████████| 20/20 [00:03<00:00,  5.30batch/s, loss=1.467]


...

Predicted: <sos> die sein $x(t)$ word geanaliseer deur 'n spektrogram te gebruik met die doel om die onmiddelike frekwensie elke 2 ms te bepaal , met 'n frekwensieresolusie van 200 hz en volle hooflob skeiding <eos>
BLEU Score: [0.273, 0.159, 0.107, 0.0]


Epoch 49/50: 100%|██████████| 20/20 [00:03<00:00,  5.06batch/s, loss=0.581]


Predicted: <sos> die sein $x(t)$ bestaan uit 'n sinusgolf met 'n onmiddelike frekwensie wat line\^er verander van $0$ hz to $20$ khz binne 2 sekondes , en dan weer terugdaal na 0 hz in die volgende 2 sekondes <eos>
BLEU Score: [0.315, 0.154, 0.097, 0.0]


Epoch 50/50: 100%|██████████| 20/20 [00:03<00:00,  5.16batch/s, loss=0.582]

Predicted: <sos> die outokorrelasie van die gevolglike sein $y[n]$ is as volg : <eos>
BLEU Score: [0.294, 0.197, 0.139, 0.09]


## Evauate on the Training set

In [8]:
EN_SRC = [' '.join(sent) for sent in english_data.data_str]
AF_REF = [[' '.join(sent)] for sent in afrikaans_data.data_str]
TRANSLATED = [translator.translate_sentence(sent) for sent in EN_SRC]
corpus_bleu(TRANSLATED, AF_REF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.3215478944083416
precisions          : [0.3215478944083416]
brevity_penalty     : 1.0
length_ratio        : 1.4775414994032765
translation_length  : 54474
reference_length    : 36868
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.2552612649659279
precisions          : [0.3215478944083416, 0.20263952750149097]
brevity_penalty     : 1.0
length_ratio        : 1.4775414994032765
translation_length  : 54474
reference_length    : 36868
******************************************************************************************
                                     BL

## Evaluate on the Test set

In [9]:
with open(f"{config.VAL_DATA}/english.txt") as data:
    english_val = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/afrikaans.txt") as data:
    afrikaans_val = data.read().strip().split("\n")

### Greedy Search

In [10]:
VAL_AF_REF = [[sent] for sent in afrikaans_val]
VAL_TRANSLATED = [translator.translate_sentence(sent) for sent in english_val]

corpus_bleu(VAL_TRANSLATED, VAL_AF_REF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.3112674832768656
precisions          : [0.3112674832768656]
brevity_penalty     : 1.0
length_ratio        : 1.5712530712530712
translation_length  : 23022
reference_length    : 14652
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.24842503117684694
precisions          : [0.3112674832768656, 0.1982699749601639]
brevity_penalty     : 1.0
length_ratio        : 1.5712530712530712
translation_length  : 23022
reference_length    : 14652
******************************************************************************************
                                     BL

### Beam search

In [11]:
VAL_TRANSLATED = [translator.translate_sentence(sent, method="beam", beam_width=3) for sent in english_val]
corpus_bleu(VAL_TRANSLATED, VAL_AF_REF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.2312174559880982
precisions          : [0.2312174559880982]
brevity_penalty     : 1.0
length_ratio        : 2.202020202020202
translation_length  : 32264
reference_length    : 14652
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.17975349452155265
precisions          : [0.2312174559880982, 0.13974428814048132]
brevity_penalty     : 1.0
length_ratio        : 2.202020202020202
translation_length  : 32264
reference_length    : 14652
******************************************************************************************
                                     BLE

## Evaluate on the SUN validation set only

In [12]:
with open(f"{config.VAL_DATA}/sun_english.txt") as data:
    sun_english_val = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/sun_afrikaans.txt") as data:
    sun_afrikaans_val = data.read().strip().split("\n")

### Greedy Search

In [13]:
SUN_VAL_AF = [[sent] for sent in sun_afrikaans_val]
SUN_VAL_TRANSLATED = [translator.translate_sentence(sent) for sent in sun_english_val]
corpus_bleu(SUN_VAL_TRANSLATED, SUN_VAL_AF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.3702854230377166
precisions          : [0.3702854230377166]
brevity_penalty     : 1.0
length_ratio        : 1.02884111169376
translation_length  : 3924
reference_length    : 3814
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.2815351820871814
precisions          : [0.3702854230377166, 0.2140566541956173]
brevity_penalty     : 1.0
length_ratio        : 1.02884111169376
translation_length  : 3924
reference_length    : 3814
******************************************************************************************
                                     BLEU-3     

### Beam Search

In [14]:
SUN_VAL_TRANSLATED = [translator.translate_sentence(sent, method="beam", beam_width=3) for sent in sun_english_val]
corpus_bleu(SUN_VAL_TRANSLATED, SUN_VAL_AF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.28151408450704224
precisions          : [0.28151408450704224]
brevity_penalty     : 1.0
length_ratio        : 1.4892501310959623
translation_length  : 5680
reference_length    : 3814
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.20377838845105983
precisions          : [0.28151408450704224, 0.14750818479447073]
brevity_penalty     : 1.0
length_ratio        : 1.4892501310959623
translation_length  : 5680
reference_length    : 3814
******************************************************************************************
                                     BL

In [15]:
metric = evaluate.load("bleu")
predictions = [translator.translate_sentence(sent, method="beam", beam_width=5) for sent in sun_english_val[10:20]]
labels = SUN_VAL_AF[10:20]
for source, pred, lab in zip(sun_english_val[10:20],predictions, labels):
    print(f"Source    : {source}")
    print(f"Prediction: {pred[:150]}")
    print(f"Label     : {lab[0][:150]}")
    print(f"BLEU      : {metric.compute(predictions=[pred], references=lab)['bleu']}")
    print()

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


Source    : <sos> component <eos>
Prediction: <sos> wat is die effek as die pole te naby aan die eenheidssirkel geplaas word <eos>
Label     : <sos> komponent <eos>
BLEU      : 0.0

Source    : <sos> architecture <eos>
Prediction: <sos> wat is die effek as die pole te naby aan die eenheidssirkel geplaas word <eos>
Label     : <sos> argitektuur <eos>
BLEU      : 0.0

Source    : <sos> specification <eos>
Prediction: <sos> wat is die effek as die pole te naby aan die eenheidssirkel geplaas word <eos>
Label     : <sos> spesifikasies <eos>
BLEU      : 0.0

Source    : <sos> at which stage of the design process would we choose the communication protocol between subsystems <eos>
Prediction: <sos> die sein $x(t)$ word geanaliseer deur 'n spektrogram te gebruik met die doel om die onmiddelike frekwensie elke 2 ms te bepaal , met 'n frekwens
Label     : <sos> by watter stap van die ontwerpsproses word die kommunikasie-kanaal tussen substelsels gekies <eos>
BLEU      : 0.0

Source    : <sos> mot